In [1]:
# Global Variables set by user **change these values before running**

# URL of the portal with the template content
PROJECT_PORTAL = "https://envisioning.maps.arcgis.com"

# username for project portal log-on (admin)
PROJECT_USERNAME = "GBushongENV"

# template content groups that will be shared with delivery org
PROJECT_GROUP_IDS = ["4d7ff4f81d6340428ef290b7de801204", "c7235e224da3441594a701bb4485528a"]

# the word that will be appended to the front to distinguish these copies of the template items and groups
COPY_PREFIX = "ACME"

In [2]:
# import libraries
from arcgis.gis import GIS
from IPython.display import display
from arcgis.gis import Group
from arcgis.gis import Item
import tempfile

In [3]:
# connection to Project organization and setting naming conventions
print("Connecting to {} with username {}...".format(PROJECT_PORTAL, PROJECT_USERNAME))
project = GIS(PROJECT_PORTAL, PROJECT_USERNAME)

FOLDER_SUFFIX = "Content"
GROUP_NAME_STRUCTURE = COPY_PREFIX + "-{}"
NAME_STRUCTURE = COPY_PREFIX + " {}"
FOLDER = NAME_STRUCTURE.format(FOLDER_SUFFIX)

Connecting to https://envisioning.maps.arcgis.com with username GBushongENV...
Enter password: ········


In [4]:
# Creation of functions to be used later
def search_by_title(target, title, obj_type):
    """search org for an existing item with title
    args:
    target -- target GIS to search
    title -- item or group to search
    obj_type -- can be "item" or "group"
    """
    try:
        if(obj_type == "group"):
            s_items = target.groups.search(title)
        elif(obj_type == "item"):
            s_items = target.content.search(query='title:{}'.format(title))
        for s_item in s_items:
            if s_item.title == title:
                return s_item
        return None
    except Exception as e:
        print("Search by title failed with args {}, {} and {}: {}".format(target, title, obj_type, e))

def modify_item(item, target):
    """Rename cloned items"""
    try:
        title = NAME_STRUCTURE.format(item.title)
        while search_by_title(target, title, "item"):
            title = input("Title `{0}` for ITEM `{1}` already exists \nNew title: ".format(title, item.title))
        return {"title": title}
    except Exception as e:
        print("Modify Item failed with args {} and {}: {}".format(item, target, e))

def modify_group(expected_title, target):
    """Rename cloned groups"""
    try:
        title = GROUP_NAME_STRUCTURE.format(expected_title)
        while search_by_title(target, title, "group"):
            title = input("Title `{0}` for GROUP `{1}` already exists \nNew title: ".format(title, expected_title))
        return title
    except Exception as e:
        print("Modify Group failed with args {} and {}: {}".format(expected_title, target, e))

def clone_group_modify(group, target):
    """Clone group and call renaming function"""
    try:
        new_group_title = modify_group(group.title, target)
        with tempfile.TemporaryDirectory() as temp_dir:
            thumbnail_file = group.download_thumbnail(temp_dir)
            target_group = target.groups.create(title=new_group_title, tags=group.tags,
                                                description=group.description, snippet=group.snippet,
                                                access=group.access, thumbnail=thumbnail_file,
                                                is_invitation_only=True, sort_field='avgRating',
                                                sort_order='asc', is_view_only=False)
            display(target_group)
        return target_group
    except Exception as e:
        print('Group {} could not be created: {}'.format(new_group_title, e))
        return None
        
def clone_items_modify(group, target, **kwargs):
    """Clone groups and items to a target GIS
    * Abstraction over arcgis.gis.ContentManager.clone_items that also renames items and groups

    args:
    group -- group containing items to be cloned. cloned group name based off 'GROUP_NAME_STRUCTURE'
    target -- target gis where items or groups will be cloned
    **kwargs:
    all additional args go into the gis.ContentManager.clone_items function described here:
    https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html?highlight=clone_items#arcgis.gis.ContentManager.clone_items
    """
    try:
        # clone the group
        new_group = clone_group_modify(group, target)
        group_map = {group.id: new_group.id} # put cloned items in the new group just created

        # clone items in the specified group to the folder
        items = [group]
        results = target.content.clone_items(items, group_mapping=group_map, **kwargs) 

        # update each result
        for result in results:
            if isinstance(result, Item):
                props = modify_item(result, target)
                args = {}
                args['data'] = props.pop('data', None)
                args['thumbnail'] = props.pop('thumbnail', None)
                args['metadata'] = props.pop('metadata', None)
                args['item_properties'] = props
                result.update(**args)
            else:
                print(result)
        return results
    except Exception as e:
        print("Clone Items modify failed with arguments {}, and {}: {}".format(group, target, e))

In [5]:
# get and print template project groups to copy in the project org
project_groups = [Group(project, ids) for ids in PROJECT_GROUP_IDS]

for group in project_groups:
    display(group)
    print(group.content())

<Group title:"aec-test-group" owner:GBushongENV>

[<Item title:"Sample Feature Layer" type:Feature Layer Collection owner:GBushongENV>, <Item title:"Sample Map" type:Web Map owner:GBushongENV>, <Item title:"Sample Scene" type:Web Scene owner:GBushongENV>]


<Group title:"aec-test-group-2" owner:GBushongENV>

[<Item title:"Sample Locator" type:Geocoding Layer owner:GBushongENV>]


In [6]:
# copy and rename project groups in project org (create external-facing version)
for group in project_groups:
    clone_res = clone_items_modify(group, project, copy_data=False, search_existing_items=False, folder=FOLDER)
    display(clone_res)

Title `ACME-aec-test-group` for GROUP `aec-test-group` already exists 
New title: ACME-aec-test-group-1
Title `ACME-aec-test-group-1` for GROUP `aec-test-group` already exists 
New title: ACME-aec-test-group-2
Title `ACME-aec-test-group-2` for GROUP `aec-test-group` already exists 
New title: ACME-aec-test


<Group title:"ACME-aec-test" owner:GBushongENV>

Title `ACME Sample Map` for ITEM `Sample Map` already exists 
New title: SM
Title `ACME Sample Feature Layer` for ITEM `Sample Feature Layer` already exists 
New title: SFL
Title `ACME Sample Scene` for ITEM `Sample Scene` already exists 
New title: SS


[<Item title:"SM" type:Web Map owner:GBushongENV>,
 <Item title:"SFL" type:Feature Layer Collection owner:GBushongENV>,
 <Item title:"SS" type:Web Scene owner:GBushongENV>]

Title `ACME-aec-test-group-2` for GROUP `aec-test-group-2` already exists 
New title: ACME-aec-test-2


<Group title:"ACME-aec-test-2" owner:GBushongENV>

[<Item title:"ACME Sample Locator" type:Geocoding Layer owner:GBushongENV>]